In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import os, sys

from src.util import generate_node_cargo_size_change, calculate_single_truck_deviation, ConsoleLogger
from src.util import read_pdpt_pickle, read_route_solution_PDPT, group_cycle_truck
from src.pdpt_ini_sol import solve_pdotw_mip, initialization
from src.pdpt_route_schedule import gurobi_master_cycle, greedy_fix_MP, MP_to_SP, cpo_sub, greedy_heuristic, time_checker_cluster, calculate_SP_cost, capacity_checker_cluster
import pickle
from pathlib import Path
import time

VERBOSE = 0

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/toy'


def find_key(list_dict, value):
    return  [k for item_ in list_dict for k, v in item_.items() if value in v]


In [2]:

def select_subroutes(ins, iniSol_filename, verbose):

    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(iniSol_filename, verbose = 0)

    res, truck_keys_shuffle, selected_truck, selected_cargo = initialization(ins, False, verbose)

    # load data from ins
    truck = ins['truck']
    cargo = ins['cargo']
    # edges = ins['edge']
    nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']
    selected_truck = {}
    selected_node = []
    selected_edge = {}
    selected_cargo = {}

    cargo_to_truck_assignment = [{key: list(set([v[0] for v in value]))} for key, value in cargo_route_file.items()]

    # print(cargo_to_truck_assignment)
    
    for truck_key in truck_keys_shuffle[:2]:
    # for truck_key in ['T10','T1','T19','T5','T2','T7','T16','T12','T9','T18','T8','T6','T4','T15','T3','T14','T17','T11','T13']:
    # truck list from Jason's jupyter notebook
        if verbose >0:
            print(f'========= START [PDOTW with truck {truck_key}] ========= ')

        truck_value = truck[truck_key]

        selected_truck[truck_key] = truck[truck_key]

        cargo_keys = find_key(cargo_to_truck_assignment, truck_key)
        if len(cargo_keys) > 0:
            for c_key in cargo_keys: 
                selected_cargo[c_key] = cargo[c_key]

        
        for v in selected_cargo.values():
            if v[3] not in selected_node:
                selected_node.append(v[3])
            if v[4] not in selected_node:
                selected_node.append(v[4])
        if truck_value[0] not in selected_node:
            selected_node.append(truck_value[0])
        if truck_value[1] not in selected_node:
            selected_node.append(truck_value[1])


    edges_ = list(set([(i,j) for i in selected_node for j in selected_node]))
    # selected_edge = {}
    # for i in selected_node:
    #     for j in selected_node:
    #         print(i,j)
    for i,j in edges_:
        selected_edge[(i,j)] = int(edge_shortest[(i,j)])

    # print(selected_edge)
    return selected_cargo, selected_truck, selected_node, selected_edge

In [3]:
iniSol_filename = dir_ + '/toyinitSol.txt'
pdpt_ins = read_pdpt_pickle(dir_ +'/toy.pkl', verbose = VERBOSE-1) 

subroutes = select_subroutes(pdpt_ins, iniSol_filename, VERBOSE)

In [6]:
def pdpt_route_schedule_decomposition(ins, subroutes):
    """
    The main function of PDPT decomposition
    input: ct = count
    
    output:
        infeasible_clusters
        removed_cargo_MP
        removed_cargo_SP

    Three steps to handle removed cargos:
        1. add a removed cargo to a different cluster, resolve the MP-SP
        2. add all removed cargos to an unused truck
        3. LNS
    """    


    
    truck = ins['truck']
    cargo = ins['cargo']
    # edges = ins['edges']
    node_list = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']
        
    selected_cargo, selected_truck, selected_node, selected_edge = subroutes


    # Setting parameters for clustering cargo and truck 
    runtime = 100
    count_node_cluster = 10
    conflict_obj_coefficient = 10000
    
      
    ###### Solve the MP+SP for each cluster ######
    
    time_start = time.time()
    
    infeasible_clusters = []
    removed_cargo_MP = []
    removed_cargo_SP = []
    solution_cluster = {}   


    ###### Initialize data structures for the solution of the current cluster ######

    solution_created_truck = {}

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, created_truck_all = \
    group_cycle_truck(selected_truck) 


    ###### Master Problem Solved by Gurobi MIP ######

    # no variable ordering
    # heuristic = 0.2
    obj_val_MP, runtime_MP, \
    x_sol, s_sol, z_sol, y_sol, u_sol, D_sol, Db_sol, \
    cost_truck_value, cost_travel_value, cost_transfer_value = \
    gurobi_master_cycle(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node, runtime*1)

    if obj_val_MP < 0:
        infeasible_clusters.append([selected_truck.keys()])
        selected_cargo, created_truck_all, selected_edge, selected_node, \
        selected_cargo_removed, obj_val_MP, runtime_MP, \
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol = \
        greedy_fix_MP(constant, selected_cargo, 
            created_truck_yCycle, created_truck_nCycle, created_truck_all, 
            selected_edge, selected_node, runtime)

        # find removed cargo from MP
        for c in selected_cargo_removed.keys():
            removed_cargo_MP.append([c, -1])

    
    ###### The MP should be feasible here ######


    ###### Subproblem Solved by CPO CP ######

    # convert MP solution to CP parameters
    truck_MP, truck_nodes, truck_nodes_index, cargo_in, cargo_out, \
    transfer_nodes, cargo_unload, cargo_load = \
    MP_to_SP(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol)  
    

    # Evaluating the feasibility of SP
    feasibility_SP, g_sol, h_sol, D_sol = \
    cpo_sub(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        truck_MP, truck_nodes, truck_nodes_index, 
        cargo_in, cargo_out,
        transfer_nodes, cargo_unload, cargo_load,
        runtime)
    print(feasibility_SP)
    if feasibility_SP != 'Feasible':
        # Greedy heuristic for cargo removal
        selected_removed_cargo, \
        truck_MP, truck_nodes, truck_nodes_index, \
        cargo_in, cargo_out, \
        transfer_nodes, cargo_unload, cargo_load = \
        greedy_heuristic(constant, selected_cargo, 
            created_truck_yCycle, created_truck_nCycle, created_truck_all, 
            selected_edge, selected_node,
            truck_MP, truck_nodes, truck_nodes_index,
            cargo_in, cargo_out,
            transfer_nodes, cargo_unload, cargo_load)

        # find removed cargo from SP
        for c in selected_removed_cargo:
            removed_cargo_SP.append([c, -1])
    else:
        print(cargo_unload)
        if time_checker_cluster(constant, selected_cargo, created_truck_all, 
                            selected_edge, truck_MP, truck_nodes, 
                            cargo_unload, cargo_load, g_sol, h_sol, D_sol):
            print('\nThe time constraint of cluster {} is not violated!'.format([selected_truck.keys()]))
        else:
            sys.exit()
        
    # Calculate SP costs: truck cost + traveling cost + transfer cost
    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
        truck_MP, truck_nodes, 
        cargo_in, cargo_out, transfer_nodes)


    # ###### Need to store solutions of clusters ######

    # for t in truck_MP:
    #     solution_created_truck[t] = truck[t]
    # solution_cluster[selected_truck.keys()] = \
    # [selected_cargo.copy(), solution_created_truck.copy(), selected_node.copy(), \
    #     truck_cost, travel_cost, transfer_cost, \
    #     truck_MP.copy(), truck_nodes.copy(), cargo_in.copy(), cargo_out.copy(), transfer_nodes.copy()]
    
    
    # ###### Solution Checker for each cluster ######
    
    # if capacity_checker_cluster(selected_cargo, solution_created_truck, 
    #                             truck_MP, truck_nodes, cargo_in, cargo_out):
    #     print('\nThe capacity of cluster {} is okay!'.format([selected_truck.keys()]))
    # else:
    #     sys.exit()
    
    
    # ###### Print important results before settle ######
    
    # time_mid = time.time()
    # print('\nThe time for the initial MP-SP is:', time_mid-time_start)

    # print('\nAll infeasible clusters are:')
    # print(infeasible_clusters)
    # print('\nThe removed cargos and corresponding clusters in MP are:')
    # for removed_cargo_cluster_pair in removed_cargo_MP:
    #     print(removed_cargo_cluster_pair)
    # print('\nThe removed cargos and corresponding clusters in SP are:')
    # for removed_cargo_cluster_pair in removed_cargo_SP:
    #     print(removed_cargo_cluster_pair)



In [7]:
pdpt_route_schedule_decomposition(pdpt_ins, subroutes)

Set parameter TimeLimit to value 100
Set parameter Heuristics to value 0.2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5843 rows, 2629 columns and 26674 nonzeros
Model fingerprint: 0xdd1dce3a
Variable types: 0 continuous, 2629 integer (2603 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [5e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 2301 rows and 660 columns
Presolve time: 0.07s
Presolved: 3542 rows, 1969 columns, 16507 nonzeros
Variable types: 0 continuous, 1969 integer (1943 binary)

Root relaxation: objective 6.025500e+04, 785 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 60255.0000    0   18          - 60255.0000      -     